In [ ]:
import pandas as pd
from src.prepare_data import prepare_data
from transformers import AutoTokenizer

Free Energy data

In [ ]:
input_path = "/n/fs/rnspace/projects/vertaix/MOF_Free_Energy_Prediction/data/properties/fe_atom/test.csv"
output_path = "/n/fs/rnspace/projects/vertaix/MOF-FreeEnergy/data/free_energy/test.csv"
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prepare_data(input_path, output_path, tokenizer)


In [ ]:
input_path = "/n/fs/rnspace/projects/vertaix/MOF_Free_Energy_Prediction/data/properties/fe_atom/validation.csv"
output_path = "/n/fs/rnspace/projects/vertaix/MOF-FreeEnergy/data/free_energy/validation.csv"
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prepare_data(input_path, output_path, tokenizer)

In [ ]:
input_path = "/n/fs/rnspace/projects/vertaix/MOF_Free_Energy_Prediction/data/properties/fe_atom/train.csv"
output_path = "/n/fs/rnspace/projects/vertaix/MOF-FreeEnergy/data/free_energy/train.csv"
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prepare_data(input_path, output_path, tokenizer)

Strain Enegry data

In [ ]:
input_path = "/n/fs/rnspace/projects/vertaix/MOF_Free_Energy_Prediction/data/properties/se_atom/test.csv"
output_path = "/n/fs/rnspace/projects/vertaix/MOF-FreeEnergy/data/strain_energy/test.csv"
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prepare_data(input_path, output_path, tokenizer)


In [ ]:
input_path = "/n/fs/rnspace/projects/vertaix/MOF_Free_Energy_Prediction/data/properties/se_atom/validation.csv"
output_path = "/n/fs/rnspace/projects/vertaix/MOF-FreeEnergy/data/strain_energy/validation.csv"
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prepare_data(input_path, output_path, tokenizer)


In [ ]:
input_path = "/n/fs/rnspace/projects/vertaix/MOF_Free_Energy_Prediction/data/properties/se_atom/train.csv"
output_path = "/n/fs/rnspace/projects/vertaix/MOF-FreeEnergy/data/strain_energy/train.csv"
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prepare_data(input_path, output_path, tokenizer)
